In [5]:
import numpy as np
import pandas as pd
import pystan
from scipy.stats import norm, multivariate_normal, invwishart, invgamma, bernoulli
from scipy.special import expit

from statsmodels.tsa.stattools import acf
import datetime
import sys
import os

from codebase.plot import * 
from codebase.file_utils import save_obj, load_obj
from codebase.post_process import * 

%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
np.random.seed(121)
nsim_data = 200
J = 6
K = 2
beta = np.array([[1,0], [-2, 0],[-1,0],[0,1], [0,-1], [0,2]], dtype=float)
mu = np.array([1,2,.3,-.8, 1, -1.4])
sigma_z = np.array([.9,1.2])
rho = 0.6
V = np.diag(sigma_z) @ np.array([[1,rho],[rho,1]]) @ np.diag(sigma_z)
sigma = np.array([1,1.2,.9,.8, 1, 1.4])
Sigma_e = np.diag(sigma**2)
Sigma_u = np.eye(J)*0.1**2
ee = multivariate_normal.rvs(mean = np.zeros(J), cov = Sigma_e, size=nsim_data)
uu = multivariate_normal.rvs(mean = np.zeros(J), cov = Sigma_u, size=nsim_data)
zz = multivariate_normal.rvs(mean = np.zeros(K), cov=V, size=nsim_data)
yy = mu + zz@beta.T + ee + uu
DD = bernoulli.rvs(p=expit(yy)); DD


array([[0, 1, 1, 1, 0, 1],
       [0, 1, 1, 0, 1, 0],
       [1, 1, 0, 0, 0, 0],
       ...,
       [0, 1, 0, 0, 1, 0],
       [1, 1, 0, 0, 1, 0],
       [1, 0, 1, 1, 0, 1]])

In [9]:
data = dict()
data['N'] = nsim_data
data['K'] = K
data['J'] = J
data['Sigma_e'] = Sigma_e
data['sigma'] = sigma
data['y'] = yy
data['beta'] = beta
data['e'] = ee
data['alpha'] = mu
data['Omega'] = data['beta']@ V @data['beta'].T + data['Sigma_e']
data['Omega_beta'] = data['beta']@ V @data['beta'].T
data['V'] = V
data['u'] = uu
data['D'] = DD

In [10]:
stan_data = dict(N = data['N'], K = data['K'], J = data['J'], DD = data['D'])

In [11]:
with open('./codebase/stan_code/discr/CFA/aug_hier.stan', 'r') as file:
    model_code = file.read()
print(model_code)

data {
  int<lower=1> N;
  int<lower=1> K;
  int<lower=1> J;
  int<lower=0, upper=1> DD[N, J];

}

transformed data{
  real<lower=0> c = 0.01;
  vector[J] zeros = rep_vector(0, J);
  cov_matrix[J] I_c = diag_matrix(rep_vector(c, J));
}

parameters {
  vector[J] alpha;
  matrix[K,K] beta_t;
  cholesky_factor_cov[K] V_chol;
  matrix[N,K] zz;
  matrix[N,J] uu;
  cov_matrix[J] Sigma_u;
}

transformed parameters{
  matrix[J,K] beta;
  matrix[N,J] yy;
  for (j in 1:J){
    for (k in 1:K) beta[j,k] = 0;
  }
  beta[1,1] = 1;
  beta[2:3,1] = beta_t[1:2,1];
  beta[4,2] = 1;
  beta[5:6,2] = beta_t[1:2,2];
  for (n in 1:N){
    yy[n,] = to_row_vector(alpha) + zz[n,] * beta' + uu[n,];
  }
  
}

model {
  to_vector(beta_t) ~ normal(0, 1);
  to_vector(alpha) ~ normal(0, 1);
  V_chol ~ lkj_corr_cholesky(2);
  for (n in 1:N){
      to_vector(zz[n, ]) ~ multi_normal_cholesky(rep_vector(0, K), V_chol);
  }
  Sigma_u ~ inv_wishart(J+6, I_c);
  for (n in 1:N){
    to_vector(uu[n,]) ~ multi_normal(zeros, Si

In [12]:
sm = pystan.StanModel(model_code=model_code, verbose=False)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_4ce62b3b12a93e6f8d2f738a4c956f37 NOW.


In [13]:
nowstr = datetime.datetime.now().strftime('%Y%m%d_%H%M%S_') # ISO 8601 format
task_id = 'CFA_IRT_aug_hier'
log_dir =  "./log/"+nowstr+"%s/" % task_id

In [14]:
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [15]:
fit_opt = sm.optimizing(data=stan_data)

In [16]:
fit_opt['beta']

array([[ 1.        ,  0.        ],
       [ 6.33816326,  0.        ],
       [ 1.20472795,  0.        ],
       [ 0.        ,  1.        ],
       [ 0.        , -4.4401342 ],
       [ 0.        ,  1.37079271]])

In [17]:
num_chains = 1
num_samples = 1000
num_warmup = 1000
num_iter = num_samples + num_warmup

In [18]:
fit_run = sm.sampling(data=stan_data, iter=num_iter, chains=num_chains)

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


In [19]:
save_obj(sm, 'sm', log_dir)
save_obj(fit_run, 'fit', log_dir)
fit=fit_run

/Users/itemgmt/myPhD/factor-analysis/src/codebase/file_utils.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


In [7]:
# log_dir = "./log/20190416_102359_CFA_NN_aug/"
# sm = load_obj('sm', log_dir)
# fit = load_obj('fit', log_dir)



In [21]:
param_names = ['Omega_beta', 'beta', 'V','alpha']

stan_samples= fit.extract(permuted=False, pars=param_names)  # return a dictionary of arrays

if num_chains ==1:
    ps = dict()
    for name in param_names:
        ps[name] = np.squeeze(stan_samples[name])
else: 
    ps = stan_samples

In [23]:
%%opts Curve {+axiswise} [width=600, height=200, tools=['hover']] 
plots = []
for i in range(data['J']):
    for j in range(data['J']):
            plots.append(plot_trace(ps['Omega_beta'][:,i,j],
                     true_value=data['Omega_beta'][i,j],
                     title = 'Posterior distribution for Omega_beta(%s,%s)'%(i,j)))
layout = hv.Layout(plots)
layout.cols(1)

:Layout
   .Overlay.Posterior_distribution_for_Omega_beta_left_parenthesis_0_comma_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_beta_left_parenthesis_0_comma_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_beta_left_parenthesis_0_comma_2_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_beta_left_parenthesis_0_comma_3_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_beta_left_parenthesis_0_comma_4_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_beta_left_parenthesis_0_comma_5_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_beta_left_parenthesis_1_comma_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_beta_left_parenthesis_1_comma_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_beta_left_parenthesis_1_comma_2_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_beta_left_parenthesis_1_comma_3_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_beta_left_parenthesis_1_comma_4_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_beta_left_parenthesis_1_comma_5_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_beta_left_parenthesis_2_comma_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_beta_left_parenthesis_2_comma_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_beta_left_parenthesis_2_comma_2_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_beta_left_parenthesis_2_comma_3_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_beta_left_parenthesis_2_comma_4_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_beta_left_parenthesis_2_comma_5_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_beta_left_parenthesis_3_comma_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_beta_left_parenthesis_3_comma_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_beta_left_parenthesis_3_comma_2_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
  

In [24]:
%%opts Curve {+axiswise} [width=600, height=200, tools=['hover']] 
plots = []
for i in range(data['J']):
    for j in range(data['J']):
            plots.append(plot_trace(acf(ps['Omega_beta'][:,i,j]),
                     title = 'Autocorrelation of Omega(%s,%s)'%(i,j)))
layout = hv.Layout(plots)
layout.cols(1)

:Layout
   .Curve.Autocorrelation_of_Omega_left_parenthesis_0_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_0_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_0_comma_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_0_comma_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_0_comma_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_0_comma_5_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_1_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_1_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_1_comma_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_1_comma_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_1_comma_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_1_comma_5_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_2_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_2_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_2_comma_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_2_comma_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_2_comma_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_2_comma_5_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_3_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_3_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_3_comma_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_3_comma_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_3_comma_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_3_comma_5_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_4_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_4_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_4_comma_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_4_comma_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_4_comma_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_4_comma_5_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_5_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_5_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_5_comma_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_5_comma_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_5_comma_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_5_comma_5_right_parenthesis :Curve   [x]   (y)

In [25]:
%%opts Curve {+axiswise} [width=600, height=200, tools=['hover']] 
plots = []
for i in range(data['J']):
            plots.append(plot_trace(ps['alpha'][:,i],
                     true_value=data['alpha'][i],
                     title = 'Posterior distribution for mu(%s)'%(i)))
layout = hv.Layout(plots)
layout.cols(1)

:Layout
   .Overlay.Posterior_distribution_for_mu_left_parenthesis_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_mu_left_parenthesis_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_mu_left_parenthesis_2_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_mu_left_parenthesis_3_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_mu_left_parenthesis_4_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_mu_left_parenthesis_5_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)

In [26]:
%%opts Curve {+axiswise} [width=600, height=200, tools=['hover']] 
plots = []
for i in range(data['J']):
            plots.append(plot_trace(acf(ps['alpha'][:,i]),
                     title = 'Autocorrelation of alpha(%s)'%(i)))
layout = hv.Layout(plots)
layout.cols(1)

:Layout
   .Curve.Autocorrelation_of_alpha_left_parenthesis_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_alpha_left_parenthesis_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_alpha_left_parenthesis_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_alpha_left_parenthesis_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_alpha_left_parenthesis_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_alpha_left_parenthesis_5_right_parenthesis :Curve   [x]   (y)

In [28]:
%%opts Curve {+axiswise} [width=600, height=200, tools=['hover']] 
plots = []
for i in range(data['K']):
    for j in range(data['K']):
            plots.append(plot_trace(ps['V'][:,i,j],
                     true_value=data['V'][i,j],
                     title = 'Posterior distribution for V(%s,%s)'%(i,j)))
layout = hv.Layout(plots)
layout.cols(1)

:Layout
   .Overlay.Posterior_distribution_for_V_left_parenthesis_0_comma_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_V_left_parenthesis_0_comma_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_V_left_parenthesis_1_comma_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_V_left_parenthesis_1_comma_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)